In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

def carregar_dados(caminho_arquivo):
    """Carrega o dataset bruto do arquivo CSV."""
    return pd.read_csv(caminho_arquivo)

def realizar_feature_engineering(df):
    """
    Aplica as transformações de colunas e criação de novas variáveis.
    Inclui: One-Hot Encoding do arrival_mode e criação do alerta de oxigênio.
    """
    # One-Hot Encoding: Cria colunas como arrival_ambulance, arrival_walk_in, etc.
    df_processado = pd.get_dummies(df, columns=['arrival_mode'], prefix='arrival')
    
    # Criar Feature Auxiliar baseada em regra clínica
    df_processado['low_oxygen_alert'] = (df_processado['oxygen_saturation'] < 92).astype(int)
    
    return df_processado

def dividir_dados(df, target_col='triage_level'):
    """Separa os dados em atributos (X) e alvo (y) com divisão estratificada."""
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    return train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

def escalonar_features(X_train, X_test, colunas_numericas):
    """Aplica a padronização (Z-score) nas colunas numéricas."""
    scaler = RobustScaler()
    
    # Garantindo que operamos em cópias para evitar avisos do Pandas
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    X_train_scaled[colunas_numericas] = scaler.fit_transform(X_train[colunas_numericas])
    X_test_scaled[colunas_numericas] = scaler.transform(X_test[colunas_numericas])
    
    return X_train_scaled, X_test_scaled

def salvar_datasets(X_train, X_test, y_train, y_test, diretorio_saida="../data/processed/"):
    """Exporta os arquivos finais para a pasta de dados processados."""
    X_train.to_csv(f"{diretorio_saida}X_train.csv", index=False)
    X_test.to_csv(f"{diretorio_saida}X_test.csv", index=False)
    y_train.to_csv(f"{diretorio_saida}y_train.csv", index=False)
    y_test.to_csv(f"{diretorio_saida}y_test.csv", index=False)

def executar_pipeline_preprocessamento():
    """Função principal que coordena todas as etapas do pré-processamento."""
    # Definição de caminhos e colunas
    ARQUIVO_ENTRADA = "../data/raw/synthetic_medical_triage.csv"
    COLUNAS_NUMERICAS = [
        'age', 'heart_rate', 'systolic_blood_pressure', 
        'oxygen_saturation', 'body_temperature', 'pain_level', 
        'chronic_disease_count', 'previous_er_visits'
    ]

    # Passo 1: Carga
    df = carregar_dados(ARQUIVO_ENTRADA)

    # Passo 2: Engenharia de Features
    df = realizar_feature_engineering(df)

    # Passo 3: Divisão (Holdout)
    X_train, X_test, y_train, y_test = dividir_dados(df)

    # Passo 4: Escalonamento
    X_train, X_test = escalonar_features(X_train, X_test, COLUNAS_NUMERICAS)

    # Passo 5: Persistência
    salvar_datasets(X_train, X_test, y_train, y_test)
    
    print("Processamento concluído com sucesso e arquivos salvos!")


executar_pipeline_preprocessamento()

Processamento concluído com sucesso e arquivos salvos!


1. Engenharia de Atributos (Feature Engineering)

A etapa de engenharia de atributos foi desenhada para transformar os dados brutos em representações que facilitem o aprendizado do modelo, incorporando regras de negócio do domínio médico e tratando limitações estatísticas das variáveis originais. As principais intervenções realizadas foram:

1.1. Codificação de Variáveis Categóricas (One-Hot Encoding)
A variável arrival_mode (Modo de Chegada), que contém categorias nominais como "Ambulância", "Cadeira de Rodas" e "Andando", foi submetida ao processo de One-Hot Encoding. A escolha por esta técnica, em detrimento de uma codificação ordinal, baseia-se no fato de que não há uma distância matemática fixa e linear entre os modos de transporte. Ao criar colunas binárias independentes para cada modalidade, permitimos que o modelo identifique a forte correlação entre a chegada por ambulância e os níveis de alta urgência (Níveis 2 e 3) sem impor uma hierarquia numérica artificial que poderia enviesar algoritmos lineares ou baseados em distância.

1.2. Criação do Indicador de Hipóxia (Low Oxygen Alert)
Foi desenvolvida uma variável binária auxiliar denominada low_oxygen_alert, acionada quando a saturação de oxigênio (oxygen_saturation) é inferior a 92%. Esta feature atua como um "gatilho clínico" baseado em protocolos reais de triagem hospitalar. Embora a saturação seja uma variável contínua, a relevância clínica de uma variação entre 98% e 95% é mínima comparada à queda de 93% para 90% (estado de hipóxia). A criação deste alerta sinaliza explicitamente ao modelo a transição para um estado crítico, compensando a forte assimetria à esquerda da variável original e facilitando a classificação correta da classe minoritária de pacientes críticos (Nível 3).

1.3 Tratamento de Variáveis de Dor e Histórico Clínico
As variáveis pain_level, chronic_disease_count e previous_er_visits foram mantidas em seu formato original, passando apenas pelo processo de escalonamento. A análise de frequência demonstrou que essas variáveis, embora assimétricas, possuem uma amplitude limitada (máximo de 10 a 11), o que permite que modelos baseados em árvores de decisão identifiquem os padrões de recorrência e intensidade de dor sem a necessidade de agrupamentos ou transformações agressivas. Essa abordagem preserva a granularidade dos dados de histórico do paciente, fundamentais para a predição de risco em casos de doenças crônicas.

2.0 ESCALONAMENTO

Escalonamento Robusto de Atributos Numéricos
Considerando que as variáveis numéricas do dataset apresentam escalas e unidades de medida muito distintas — como a Pressão Sistólica, que atinge valores superiores a 160 mmHg, e a Temperatura Corporal, que oscila em uma faixa estreita em torno de 37 °C — aplicou-se o processo de escalonamento utilizando a técnica RobustScaler. Diferente da padronização convencional (Z-score), que utiliza a média e o desvio padrão e é altamente sensível a valores extremos, o escalonamento robusto baseia-se na mediana e no intervalo interquartil. Esta decisão é estratégica para o contexto de triagem médica, pois garante que valores extremos de sinais vitais, como picos de frequência cardíaca ou crises hipertensivas, não sejam tratados como ruídos estatísticos descartáveis, mas sim preservados como indicadores cruciais de gravidade. Ao normalizar os dados dessa forma, asseguramos que o algoritmo atribua importância a cada sinal vital com base em sua relevância clínica, sem que as diferentes magnitudes numéricas ou a presença de outliers legítimos distorçam o aprendizado do modelo.

3.0 Divisão de Dados e Amostragem Estratificada

Para a avaliação do desempenho do modelo, os dados foram particionados em conjuntos de treinamento e teste seguindo a proporção de 80% e 20%, respectivamente. A decisão mais crítica nesta etapa foi a implementação da amostragem estratificada, configurada através do parâmetro stratify. Dado que o dataset apresenta um desbalanceamento severo — onde pacientes críticos (Nível 3) representam apenas cerca de 5% do total de amostras — uma divisão puramente aleatória poderia resultar em sub-representação ou ausência total de casos graves em um dos conjuntos. A estratificação garante que a distribuição original das classes seja preservada fielmente tanto no treino quanto no teste. Isso assegura que o modelo tenha exemplos suficientes para aprender os padrões de risco extremo durante o treinamento e, simultaneamente, permite que a sua capacidade de identificar esses casos raros seja validada de forma justa e estatisticamente significativa no conjunto de teste.

OBS:

Optou-se por não aplicar técnicas de balanceamento artificial no conjunto de dados, como o sobreajuste sintético (oversampling via SMOTE) ou a redução de amostras (undersampling). Esta decisão baseia-se na premissa de que a distribuição original das classes — onde casos de baixa urgência predominam (55%) e casos críticos são raros (5%) — constitui uma característica intrínseca e valiosa do domínio de triagem hospitalar. Alterar essas proporções artificialmente durante o pré-processamento poderia distorcer a realidade operacional que o modelo encontrará em produção, levando-o a uma percepção irreal sobre a frequência de eventos graves e, consequentemente, aumentando a taxa de falsos alarmes.